In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from sklearn import datasets
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay, f1_score

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset




In [3]:
def fn_plot_tf_hist(hist_df : pd.DataFrame):
    '''
    Note this function is specifically designed to plot Tensorflow training output
    Args:
      hist_df : pandas DataFrame with four columns
                For 'x' values, we will use index
                first column is accuracy
                Second column is loss
                third column is val_accuracy
                fourth column is val_loss
    '''
    fig, axes = plt.subplots(1,2 , figsize = (15,6)) # instantiate plot

    # properties  matplotlib.patch.Patch 
    props = dict(boxstyle='round', facecolor='aqua', alpha=0.4)
    facecolor = 'cyan'
    fontsize=12
    
    # Get columns by index to eliminate any column naming error
    y1 = hist_df.columns[0]
    y2 = hist_df.columns[1]
    y3 = hist_df.columns[2]
    y4 = hist_df.columns[3]

    # Where was min loss
    best = hist_df[hist_df[y4] == hist_df[y4].min()]
 
    ax = axes[0]

    hist_df.plot(y = [y2,y4], ax = ax, colormap=CMAP)


    # little beautification
    txtFmt = "Loss: \n  train: {:6.4f}\n   test: {:6.4f}"
    txtstr = txtFmt.format(hist_df.iloc[-1][y2],
                           hist_df.iloc[-1][y4]) #text to plot
    
    # place a text box in upper middle in axes coords
    ax.text(0.3, 0.95, txtstr, transform=ax.transAxes, fontsize=fontsize,
            verticalalignment='top', bbox=props)

    # Mark arrow at lowest
    ax.annotate(f'Min: {best[y4].to_numpy()[0]:6.4f}', # text to print
                xy=(best.index.to_numpy(), best[y4].to_numpy()[0]), # Arrow start
                xytext=(best.index.to_numpy()-1, best[y4].to_numpy()[0]), # location of text 
                fontsize=fontsize, va='bottom', ha='right',bbox=props, # beautification of text
                arrowprops=dict(facecolor=facecolor, shrink=0.05)) # arrow

    # Draw vertical line at best value
    ax.axvline(x = best.index.to_numpy(), color = 'green', linestyle='-.', lw = 3);

    ax.set_xlabel("Epochs")
    ax.set_ylabel(y2.capitalize())
    ax.set_title('Errors')
    ax.legend(loc = 'upper left') # model legend to upper left

    ax = axes[1]

    hist_df.plot( y = [y1, y3], ax = ax, colormap=CMAP)
    
    # little beautification
    txtFmt = "Accuracy: \n  train: {:6.4f}\n  test:  {:6.4f}"
    txtstr = txtFmt.format(hist_df.iloc[-1][y1],
                           hist_df.iloc[-1][y3]) #text to plot

    # place a text box in upper middle in axes coords
    ax.text(0.3, 0.2, txtstr, transform=ax.transAxes, fontsize=fontsize,
            verticalalignment='top', bbox=props)

    # Mark arrow at lowest
    ax.annotate(f'Best: {best[y3].to_numpy()[0]:6.4f}', # text to print
                xy=(best.index.to_numpy(), best[y3].to_numpy()[0]), # Arrow start
                xytext=(best.index.to_numpy()-1, best[y3].to_numpy()[0]), # location of text 
                fontsize=fontsize, va='bottom', ha='right',bbox=props, # beautification of text
                arrowprops=dict(facecolor=facecolor, shrink=0.05)) # arrow
    
    
    # Draw vertical line at best value
    ax.axvline(x = best.index.to_numpy(), color = 'green', linestyle='-.', lw = 3);

    ax.set_xlabel("Epochs")
    ax.set_ylabel(y1.capitalize())
    ax.legend(loc = 'lower left')
    
    plt.tight_layout()

In [4]:
RANDOM_STATE = 24 # for initialization ----- REMEMBER: to remove at the time of promotion to production


EPOCHS = 100 # number of cycles to run
ALPHA = 0.001 # learning rate
WEIGHT_DECAY = 0.001
BATCH_SIZE = 32
TRAIN_SIZE = BATCH_SIZE * 9
LR_FACTOR = 0.1
LR_PATIENCE= 10

# Set parameters for decoration of plots
params = {'legend.fontsize' : 'large',
          'figure.figsize'  : (9,9),
          'axes.labelsize'  : 'x-large',
          'axes.titlesize'  :'x-large',
          'xtick.labelsize' :'large',
          'ytick.labelsize' :'large',
         }

plt.rcParams.update(params) # update rcParams
CMAP = plt.cm.coolwarm
plt.style.use('seaborn-v0_8-darkgrid') # plt.style.use('ggplot')

In [5]:
data_df = pd.read_csv('ionosphere.data', header = None)
data_df.shape

(351, 35)

In [6]:
X = data_df.drop(data_df.columns[-1], axis = 1).to_numpy()

y = data_df[data_df.columns[-1]].to_numpy()

In [7]:
train_df, test_df= train_test_split(data_df,train_size=TRAIN_SIZE,stratify=data_df[data_df.columns[-1]], random_state=RANDOM_STATE)
train_df.shape, test_df.shape

((288, 35), (63, 35))

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(f'using {device} device')

using cpu device


In [9]:
''' using singleton design pattern'''

class Transformers:
    _instance = None

    def __init__(self):
        if Transformers._instance is not None:
            raise Exception ("GlobalScaler class is a singleton.")

        self.scaler  = StandardScaler()
        self.encoder = LabelEncoder()

    @classmethod
    def get_instance(cls):
        if cls._instance is None:
            cls._instance = Transformers()
        return cls._instance

In [10]:
'''
In this code , we define a custom dataset called FifaDS that takes a Dataframe as input.
'''

'\nIn this code , we define a custom dataset called FifaDS that takes a Dataframe as input.\n'

In [11]:
class IonoDS(Dataset):
    transformers= Transformers.get_instance()
    
    def __init__(self,
                dataframe: pd.DataFrame,
                device: str= device,
                is_train= True,
                label_col= None
                ):

        super(IonoDS, self).__init__()

        self.df= dataframe
        self.device= device
        self.is_train = is_train
        self.encoder = self.transformers.encoder
        self.scaler = self.transformers.scaler
        self.label_col = label_col

        y = self.df[label_col].to_numpy()
        X = self.df.drop(label_col, axis = 1)

        if self.is_train:
            self.labels = self.encoder.fit_transform(y)
            self.features = self.scaler.fit_transform(X)
        else:
            self.labels = self.encoder.transform(y)
            self.labels = self.encoder.transform(X)
    def __len__(self):
         return len(self.features)
    def __getitem__(self, index):
         features = self.features[index]
         label = self.labels[index]

         features = torch.tensor(features, dtype=torch.float32 , device=self.device)
         label = torch.tensor(label, dtype = torch.int64, device = self.device)


         return features, label

In [12]:
class Model(nn.Module):
    def __init__(self, input_dim):

        super(Model, self).__init__()

        dor1 = 0.05
        dor2 = 0.15
        dor3 = 0.25

        self.layer1 = nn.Linear(input_dim, 26)
        self.bm1    = nn.BatchNorm1d(26)
        self.do1    = nn.Dropout(dor1)
        self.act1   = nn.ReLU()

        self.layer2 = nn.Linear(26,18)
        self.bm2    = nn.BatchNorm1d(18)
        self.do2    = nn.Dropout(dor2)
        self.act2   = nn.ReLU()

        self.layer3 = nn.Linear(18,10)
        self.bm3    = nn.BatchNorm1d(10)
        self.do3    = nn.Dropout(dor3)
        self.act3   = nn.ReLU()

        self.layer4  = nn.Linear(10,2)
        self.softmax = nn.LogSoftmax(dim = 1)

    def forward(self, x):

        #activation functions by layer

        x = self.do1(self.act1(self.bn1(self.layer1(x))))
        x = self.do2(self.act2(self.bn2(self.layer2(x))))
        x = self.do3(self.act3(self.bn3(self.layer3(x))))

        output = self.softmax(self.layer4(x))

        return output

input_dim = 34
model     = Model(input_dim).to(device)
        
print(model)
        

Model(
  (layer1): Linear(in_features=34, out_features=26, bias=True)
  (bm1): BatchNorm1d(26, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (do1): Dropout(p=0.05, inplace=False)
  (act1): ReLU()
  (layer2): Linear(in_features=26, out_features=18, bias=True)
  (bm2): BatchNorm1d(18, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (do2): Dropout(p=0.15, inplace=False)
  (act2): ReLU()
  (layer3): Linear(in_features=18, out_features=10, bias=True)
  (bm3): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (do3): Dropout(p=0.25, inplace=False)
  (act3): ReLU()
  (layer4): Linear(in_features=10, out_features=2, bias=True)
  (softmax): LogSoftmax(dim=1)
)


In [13]:
label_col = 34
train_ds  = IonoDS(train_df, is_train= True, label_col = label_col)
test_ds   = IonoDS(test_df,  is_train=False, label_col = label_col)

ValueError: y should be a 1d array, got an array of shape (63, 34) instead.

In [ ]:
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
test_loader  = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle = True)

In [ ]:
loss_fn     = nn.CrossEntropyLoss()

loss, tloss = [],[]
acc, tacc   = [],[]
n_epoch     = []

optimizer = torch.optim.Adam(model.parameters(),
                            lr = ALPHA,
                            weight_decay = 0.1e-5)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                      mode = 'min',
                                                      factor = LR_FACTOR,
                                                      patience = LR_PATIENCE,
                                                      min_lr= 1e-5)

minLoss  = float('inf')

savePath = os.path.join(modelDir, subDir, 'iono.pth')

for epoch in range(EPOCHS):

    train_loss = 0.0
    train_acc  = 0.0

    for i, data in enumerate(train_loader):

        inputs, labels = data

        optimizer.zero_grad()
        outputs    = model(inputs)
        preds      = torch.argmax(outputs, dim = 1)
        batch_loss = loss_fn(outputs, labels)
        batch_acc  = accuracy_score(labels.cpu().numpy(), preds.cpu().numpy())
        batch_loss.backward()
        optimizer.step()

        train_loss += batch_loss.item() * inputs.size(0)
        train_acc  ++ batch_acc * inputs.size(0)

    train_loss /= len(train_ds)
    train_acc  /= len(train_ds)

    loss.append(train_loss)
    acc.append(train_acc)